# Descriptor
The descriptor protocol is define below. Data descriptors have the **'\_\_set__'** protocol defined, whereas the non data do not.
Data descriptors take precedence over instance name space. Under the hood, properties, classmethod, staticmethods and slots are in fact descriptors.

In [2]:
class Descriptor:
    
    def __init__(self):
        pass

    def __set_name__(self, owner, name):
        pass
        
    def __get__(self, instance, owner):
        pass

    def __set__(self, instance, value):
        pass

    def __delete__(self, instance):
        pass

### Implement with descriptor

In [4]:
class ValidAmount:

    def __init__(self, max_amount):
        self.max_amount = max_amount

    def __set_name__(self, owner, name):
        self.name = name # set name attribute within instance object

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__.get(f"valid_{self.name}")

    def __set__(self, instance, value):
        if not isinstance(value, (float, int)):
            raise TypeError(f"{value} must be int or float")

        if value < 0:
            raise ValueError(f"{value} must be positive")

        if value >  self.max_amount:
            raise ValueError(f"{value} should me less than {self.amount}")

        instance.__dict__[f"valid_{self.name}"] = value

    def __delete__(self, instance):
        del instance.__dict__[f"valid_{self.name}"]


In [5]:
class BankAccount:
    deposit = ValidAmount(100000)
    withdraw = ValidAmount(5000)

    def __init__(self, owner):
        self.owner = owner

In [6]:
b = BankAccount("Nicolas")
b.__dict__

{'owner': 'Nicolas'}

In [7]:
b.deposit = 8000
b.withdraw = 588
b.__dict__

{'owner': 'Nicolas', 'valid_deposit': 8000, 'valid_withdraw': 588}

In [8]:
b.deposit

8000

In [9]:
try:
    b.deposit = -5
except ValueError as err:
    print(err)

-5 must be positive


### Implement with properties

In [11]:
class BankAccount:

    def __init__(self, max_amount):
        self.max_amount = max_amount
        self.deposit = 0

    @property
    def deposit(self):
        return self._valid_deposit

    @deposit.setter
    def deposit(self, value):
        if not isinstance(value, (float, int)):
            raise TypeError(f"{value} must be int or float")

        if value < 0:
            raise ValueError(f"{value} must be positive")

        if value >  self.max_amount:
            raise ValueError(f"{value} should me less than {self.amount}")

        self._valid_deposit = value

    @deposit.deleter
    def deposit(self):
        del self._valid_deposit

In [12]:
new_b = BankAccount(500)

In [13]:
new_b.__dict__

{'max_amount': 500, '_valid_deposit': 0}

In [14]:
class BankAccount:

    def __init__(self, max_amount):
        self.max_amount = max_amount
        self.deposit = 0

    def get_deposit(self):
        return self._valid_deposit

    def set_deposit(self, value):
        if not isinstance(value, (float, int)):
            raise TypeError(f"{value} must be int or float")

        if value < 0:
            raise ValueError(f"{value} must be positive")

        if value >  self.max_amount:
            raise ValueError(f"{value} should me less than {self.amount}")

        self._valid_deposit = value

    def del_deposit(self):
        del self._valid_deposit

    deposit = property(fget = get_deposit, fset = set_deposit, fdel = del_deposit)

In [15]:
new_b = BankAccount(500)

In [16]:
new_b.__dict__

{'max_amount': 500, '_valid_deposit': 0}

In [17]:
new_b.__dict__

{'max_amount': 500, '_valid_deposit': 0}

In [18]:
# functions can be defined outside class 
# but it is a bad practice 
# here account refer to BankAccount instance and in class definition self refer to the instance

def get_deposit(account):
    return account._valid_deposit

def set_deposit(account, value):
    if not isinstance(value, (float, int)):
        raise TypeError(f"{value} must be int or float")

    if value < 0:
        raise ValueError(f"{value} must be positive")

    if value >  account.max_amount:
        raise ValueError(f"{value} should me less than {self.amount}")

    account._valid_deposit = value

def del_deposit(account):
    del account._valid_deposit

In [19]:
class BankAccount:

    def __init__(self, max_amount):
        self.max_amount = max_amount
        self.deposit = 0
        
    deposit = property(fget = get_deposit, fset = set_deposit, fdel = del_deposit)

In [20]:
new_b = BankAccount(500)

In [21]:
new_b.__dict__

{'max_amount': 500, '_valid_deposit': 0}